# Implementing DCA in pytorch

Source CC function in [seqmodel](https://github.com/sokrypton/seqmodels/blob/master/seqmodels.ipynb)
Reimplemeting tensorflow code in pytorch

In [ ]:
import numpy as np
import torch

import read_config
from dataloader import MSADataset, OneHotTransform

In [9]:
config = read_config.Config("../config2d.yaml")
dataset = MSADataset(config.aligned_msa_fullpath, transform=OneHotTransform(21))
N = len(dataset)

protein_seq, weight = dataset[0]
ncat = 21
L = int(len(protein_seq) / ncat)
ncol = L

N, L

(14441, 559)

In [5]:
ncol, ncat

(559, 21)

In [8]:
bias = torch.zeros((ncol,ncat), requires_grad = True)
w = torch.zeros((ncol, ncat, ncol, ncat), requires_grad=True)

# we do not want weights between the various nodes in a given position. 
# i.e. weights between nodes (i, a) and (j, b) only exist if i not = j
# so set these weights to zero
w_eye = w * torch.reshape(1 - torch.eye(ncol), (ncol,1,ncol, 1))
# symmetrize w so that the weight between (i,a) and (j, b) is the
# same as the weight between (j, b) and (i, a)
weights = w_eye + w_eye.permute(2,3,0,1)